In [ ]:
GTID ( Global Transaction Identifier )
====================================================

-- is a unique identifier created and associated with each transaction committed in the source server.
-- if a transaction is not written to the binary log , it is not assigned a GTID.
-- replicated transactions retain the same GTID that was assigned to it on the source.
-- the mysql.gtid_executed table is used to preserve the GTIDs.
-- GTID is represented by pair of characters seperated by colon -> source_id : transaction_id
-- cde07bc2-178f-11ef-b846-000c29a50cb9:1-4
-- RESET MASTER will clear the GTIDs and clear all the data in mysql.gtid_executed table.

GTID Auto-Positioning 
============================

-- for gtid based replication you have to enable source_auto_position which is disabled by default.
-- in the initial handshake the replica sends a gtid set that it has already received.
-- the source responds by sending all transactions which is not included in the gtid set sent by replica.

In [ ]:
Change hostname of both the primary and replica servers (optional)
=======================================================================


[root@mysqlserver ~]# hostnamectl set-hostname mysqlprod01
[root@mysqlserver ~]# 
[root@mysqlserver ~]# uname -a
Linux mysqlprod01
[root@mysqlserver ~]# 

[root@MySqlServer2 ~]# hostnamectl set-hostname mysqlprod02
[root@MySqlServer2 ~]# 
[root@MySqlServer2 ~]# uname -a
Linux mysqlprod02 4.18.0-348.el8.x86_64 
[root@MySqlServer2 ~]# 

In [ ]:
Start both servers with GTID enabled 
=====================================================

In primary server 
---------------------------

[root@mysqlprod01 ~]# vi /var/lib/mysql-files/replication.cnf 
[root@mysqlprod01 ~]# 
[root@mysqlprod01 ~]# 
[root@mysqlprod01 ~]# cat /var/lib/mysql-files/replication.cnf
[mysqld]

log-bin = /var/lib/mysql/binlog/mysqlprod01_binlog
log-bin-index = /var/lib/mysql/binlog/mysqlprod01_binlog.index
binlog-format = MIXED

gtid-mode = ON
enforce-gtid-consistency = ON
[root@mysqlprod01 ~]# 


In replica server
---------------------------------

[root@mysqlprod02 ~]# vi /etc/replication.cnf 
[root@mysqlprod02 ~]# 
[root@mysqlprod02 ~]# cat /etc/replication.cnf 
[mysqld]

log-bin = /var/lib/mysql/mysqlprod02_binlog
log-bin-index = /var/lib/mysql/mysqlprod02_binlog.index
relay-log = /var/lib/mysql/relaylog/mysqlprod02_relaylog
relay-log-index = /var/lib/mysql/relaylog/mysqlprod02_relaylog.index

binlog-format = MIXED

skip-replica-start

gtid-mode = ON
enforce-gtid-consistency = ON
[root@mysqlprod02 ~]# 


[root@mysqlprod01 ~]# systemctl stop mysqld
[root@mysqlprod01 ~]# 
[root@mysqlprod01 ~]# systemctl start mysqld
[root@mysqlprod01 ~]# 

mysql> select @@hostname , user() ;
+-------------+----------------+
| @@hostname  | user()         |
+-------------+----------------+
| mysqlprod01 | root@localhost |
+-------------+----------------+
1 row in set (0.00 sec)

mysql> show master status;
+---------------------------+----------+--------------+------------------+-------------------+
| File                      | Position | Binlog_Do_DB | Binlog_Ignore_DB | Executed_Gtid_Set |
+---------------------------+----------+--------------+------------------+-------------------+
| mysqlprod01_binlog.000001 |      157 |              |                  |                   |
+---------------------------+----------+--------------+------------------+-------------------+
1 row in set (0.00 sec)

mysql> show variables like '%gtid%';
+----------------------------------+-----------+
| Variable_name                    | Value     |
+----------------------------------+-----------+
| binlog_gtid_simple_recovery      | ON        |
| enforce_gtid_consistency         | ON        |
| gtid_executed                    |           |
| gtid_executed_compression_period | 0         |
| gtid_mode                        | ON        |


mysql> show databases;
+--------------------+
| Database           |
+--------------------+
| information_schema |
| mysql              |
| performance_schema |
+--------------------+
3 rows in set (0.00 sec)



[root@mysqlprod02 ~]# systemctl stop mysqld
[root@mysqlprod02 ~]# 
[root@mysqlprod02 ~]# systemctl start mysqld
[root@mysqlprod02 ~]# 

mysql> select @@hostname , user() ;
+-------------+----------------+
| @@hostname  | user()         |
+-------------+----------------+
| mysqlprod02 | root@localhost |
+-------------+----------------+
1 row in set (0.00 sec)

mysql> show master status;
+---------------------------+----------+--------------+------------------+-------------------+
| File                      | Position | Binlog_Do_DB | Binlog_Ignore_DB | Executed_Gtid_Set |
+---------------------------+----------+--------------+------------------+-------------------+
| mysqlprod02_binlog.000001 |      157 |              |                  |                   |
+---------------------------+----------+--------------+------------------+-------------------+
1 row in set (0.01 sec)

mysql> show variables like '%gtid%';
+----------------------------------+-----------+
| Variable_name                    | Value     |
+----------------------------------+-----------+
| binlog_gtid_simple_recovery      | ON        |
| enforce_gtid_consistency         | ON        |
| gtid_executed                    |           |
| gtid_executed_compression_period | 0         |
| gtid_mode                        | ON        |

mysql> show replica status;
Empty set (0.00 sec)

mysql> show databases;
+--------------------+
| Database           |
+--------------------+
| information_schema |
| mysql              |
| performance_schema |
+--------------------+
3 rows in set (0.01 sec)

In [ ]:
Configure the replica to use GTID-based auto positioning 
======================================================================

mysql> select @@hostname , user() ;
+-------------+----------------+
| @@hostname  | user()         |
+-------------+----------------+
| mysqlprod02 | root@localhost |
+-------------+----------------+
1 row in set (0.00 sec)

mysql> change replication source to
    -> source_host = '192.168.153.133',
    -> source_port = 3306,
    -> source_user = 'replicator',
    -> source_password = 'P@ssword2024',
    -> source_auto_position = 1;
Query OK, 0 rows affected, 2 warnings (0.03 sec)


mysql> start replica;
Query OK, 0 rows affected (0.08 sec)

mysql> show replica status \G
*************************** 1. row ***************************
             Replica_IO_State: Connecting to source
                  Source_Host: 192.168.153.133
                  Source_User: replicator
                  Source_Port: 3306
                Connect_Retry: 60
             
                Last_IO_Errno: 2061
                Last_IO_Error: error connecting to master 'replicator@192.168.153.133:3306' 
                              -retry-time: 60 retries: 1 message: Authentication plugin 'caching_sha2_password' 
                               reported error: Authentication requires secure connection.
            

mysql> select @@hostname , user() ;
+-------------+----------------+
| @@hostname  | user()         |
+-------------+----------------+
| mysqlprod01 | root@localhost |
+-------------+----------------+
1 row in set (0.00 sec)

mysql> alter user replicator identified with mysql_native_password by 'P@ssword2024';
Query OK, 0 rows affected (0.00 sec)

mysql> stop replica;
Query OK, 0 rows affected (0.00 sec)

mysql> start replica;
Query OK, 0 rows affected (0.10 sec)

mysql> show replica status \G
*************************** 1. row ***************************
             Replica_IO_State: 
                  Source_Host: 192.168.153.133
                  Source_User: replicator
                  Source_Port: 3306

                Last_IO_Errno: 13117
                Last_IO_Error: Fatal error: The slave I/O thread stops because master and slave have equal MySQL 
                                            server ids; these ids must be different for replication to work 
                                            (or the --replicate-same-server-id option must be used on slave but 
                                            this does not always make sense; please check the manual before using it).

mysql> stop replica;
Query OK, 0 rows affected (0.00 sec)


mysql> select @@hostname , @@server_id ;
+-------------+-------------+
| @@hostname  | @@server_id |
+-------------+-------------+
| mysqlprod01 |           1 |
+-------------+-------------+
1 row in set (0.01 sec)

mysql> select @@hostname , @@server_id ;
+-------------+-------------+
| @@hostname  | @@server_id |
+-------------+-------------+
| mysqlprod02 |           1 |
+-------------+-------------+
1 row in set (0.00 sec)


[root@mysqlprod02 ~]# systemctl stop mysqld
[root@mysqlprod02 ~]# 
[root@mysqlprod02 ~]# vi /etc/replication.cnf 
[root@mysqlprod02 ~]# 
[root@mysqlprod02 ~]# cat /etc/replication.cnf | grep server
server-id = 2
[root@mysqlprod02 ~]# 
[root@mysqlprod02 ~]# systemctl start mysqld
[root@mysqlprod02 ~]# 

mysql> select @@hostname , @@server_id ;
+-------------+-------------+
| @@hostname  | @@server_id |
+-------------+-------------+
| mysqlprod02 |           2 |
+-------------+-------------+
1 row in set (0.00 sec)


mysql> start replica;
Query OK, 0 rows affected (0.08 sec)

mysql> show replica status \G

*************************** 1. row ***************************

             Replica_IO_State: Waiting for source to send event
                  Source_Host: 192.168.153.133
                  Source_User: replicator
                  Source_Port: 3306
                  Connect_Retry: 60
                 Source_Log_File: mysqlprod01_binlog.000001
                 Read_Source_Log_Pos: 445
                 Relay_Log_File: mysqlprod02_relaylog.000003
                 Relay_Log_Pos: 679
                 Relay_Source_Log_File: mysqlprod01_binlog.000001

                 Replica_IO_Running: Yes
                 Replica_SQL_Running: Yes

                 Source_Server_Id: 1
                 Source_UUID: cde07bc2-178f-11ef-b846-000c29a50cb9
                 Source_Info_File: mysql.slave_master_info
                
                 Replica_SQL_Running_State: Replica has read all relay log; waiting for more updates

                 Retrieved_Gtid_Set: cde07bc2-178f-11ef-b846-000c29a50cb9:1
                 Executed_Gtid_Set: cde07bc2-178f-11ef-b846-000c29a50cb9:1
                 Auto_Position: 1


In [ ]:
Run some transactions in primary and check in replica
==================================================================

root@mysqlprod01> create database employeesdb;
Query OK, 1 row affected (0.00 sec)

root@mysqlprod01> use employeesdb;
Database changed
root@mysqlprod01> create table employee_details ( emp_id int primary key , emp_name varchar(50) , emp_dob date );
Query OK, 0 rows affected (0.05 sec)

root@mysqlprod01> insert into employee_details values
    -> (1001 , 'Ranga' , now() ),
    -> (1002 , 'Manga' , now() ),
    -> (1003 , 'Changa' , now() );
Query OK, 3 rows affected, 3 warnings (0.02 sec)
Records: 3  Duplicates: 0  Warnings: 3

root@mysqlprod01> select * from employee_details;
+--------+----------+------------+
| emp_id | emp_name | emp_dob    |
+--------+----------+------------+
|   1001 | Ranga    | 2024-05-23 |
|   1002 | Manga    | 2024-05-23 |
|   1003 | Changa   | 2024-05-23 |
+--------+----------+------------+
3 rows in set (0.00 sec)

root@mysqlprod01> show master status;
+---------------------------+----------+--------------+------------------+------------------------------------------+
| File                      | Position | Binlog_Do_DB | Binlog_Ignore_DB | Executed_Gtid_Set                        |
+---------------------------+----------+--------------+------------------+------------------------------------------+
| mysqlprod01_binlog.000001 |     1355 |              |                  | cde07bc2-178f-11ef-b846-000c29a50cb9:1-4 |
+---------------------------+----------+--------------+------------------+------------------------------------------+
1 row in set (0.00 sec)


root@mysqlprod02> show replica status \G

*************************** 1. row ***************************
             Replica_IO_State: Waiting for source to send event
                  Source_Host: 192.168.153.133
                  Source_User: replicator
                  Source_Port: 3306
                Connect_Retry: 60
              Source_Log_File: mysqlprod01_binlog.000001
          Read_Source_Log_Pos: 1355
               Relay_Log_File: mysqlprod02_relaylog.000003
                Relay_Log_Pos: 1589
        Relay_Source_Log_File: mysqlprod01_binlog.000001

           Replica_IO_Running: Yes
           Replica_SQL_Running: Yes

           Retrieved_Gtid_Set: cde07bc2-178f-11ef-b846-000c29a50cb9:1-4
           Executed_Gtid_Set: cde07bc2-178f-11ef-b846-000c29a50cb9:1-4


root@mysqlprod02> select count(*) from employeesdb.employee_details;
+----------+
| count(*) |
+----------+
|        3 |
+----------+
1 row in set (0.31 sec)


In [ ]:
Suppose there is an application release and need to revert if there are any errors 
============================================================================================


stop the replica 
==============================

root@mysqlprod02> stop replica;
Query OK, 0 rows affected (0.00 sec)

root@mysqlprod02> show replica status \G
*************************** 1. row ***************************
             Replica_IO_State: 

             Replica_IO_Running: No
             Replica_SQL_Running: No



Let application team do their release in production 
===========================================================

root@mysqlprod01> call sp_insertprod_data(1);
Query OK, 1 row affected (5.93 sec)

root@mysqlprod01> show master status;
+---------------------------+----------+--------------+------------------+----------------------------------------------+
| File                      | Position | Binlog_Do_DB | Binlog_Ignore_DB | Executed_Gtid_Set                            |
+---------------------------+----------+--------------+------------------+----------------------------------------------+
| mysqlprod01_binlog.000001 |  4430042 |              |                  | cde07bc2-178f-11ef-b846-000c29a50cb9:1-10006 |
+---------------------------+----------+--------------+------------------+----------------------------------------------+
1 row in set (0.00 sec)

root@mysqlprod01> call sp_insertprod_data(10001);
Query OK, 1 row affected (5.77 sec)

root@mysqlprod01> call sp_insertprod_data(20001);
Query OK, 1 row affected (5.76 sec)

root@mysqlprod01> call sp_insertprod_data(30001);
Query OK, 1 row affected (5.61 sec)

root@mysqlprod01> show master status;
+---------------------------+----------+--------------+------------------+----------------------------------------------+
| File                      | Position | Binlog_Do_DB | Binlog_Ignore_DB | Executed_Gtid_Set                            |
+---------------------------+----------+--------------+------------------+----------------------------------------------+
| mysqlprod01_binlog.000001 | 17780042 |              |                  | cde07bc2-178f-11ef-b846-000c29a50cb9:1-40006 |
+---------------------------+----------+--------------+------------------+----------------------------------------------+
1 row in set (0.01 sec)


Suppose the user has accidently dropped the table during the release 
=========================================================================

root@mysqlprod01> drop table products;
Query OK, 0 rows affected (0.22 sec)

root@mysqlprod01> select count(*) from employeesdb.products;
ERROR 1146 (42S02): Table 'employeesdb.products' doesn't exist
root@mysqlprod01> 

root@mysqlprod01> show master status;
+---------------------------+----------+--------------+------------------+----------------------------------------------+
| File                      | Position | Binlog_Do_DB | Binlog_Ignore_DB | Executed_Gtid_Set                            |
+---------------------------+----------+--------------+------------------+----------------------------------------------+
| mysqlprod01_binlog.000001 | 17780266 |              |                  | cde07bc2-178f-11ef-b846-000c29a50cb9:1-40007 |
+---------------------------+----------+--------------+------------------+----------------------------------------------+
1 row in set (0.00 sec)


Find the GTID of the drop table statement 
=====================================================

[root@mysqlprod01 ~]# mysqlbinlog /var/lib/mysql/binlog/mysqlprod01_binlog.000001 > 
/home/mysqluser/backups/mysqlprod01_binlog_1.sql
[root@mysqlprod01 ~]# 

SET @@SESSION.GTID_NEXT= 'cde07bc2-178f-11ef-b846-000c29a50cb9:40007'/*!*/;
# at 17780119
#240523 14:27:34 server id 1  end_log_pos 17780266 CRC32 0xa3a6868d     Query   thread_id=19    exec_time=1     
error_code=0    Xid = 120132
SET TIMESTAMP=1716454654/*!*/;
SET @@session.pseudo_thread_id=19/*!*/;
DROP TABLE `products` /* generated by server */
/*!*/;
SET @@SESSION.GTID_NEXT= 'AUTOMATIC' /* added by mysqlbinlog */ /*!*/;
DELIMITER ;
# End of log file


In replica start replica until that GTID 
=======================================================

root@mysqlprod02> show replica status \G
*************************** 1. row ***************************
             Replica_IO_State: 
                  Source_Host: 192.168.153.133

           Replica_IO_Running: No
          Replica_SQL_Running: No
        
           Retrieved_Gtid_Set: cde07bc2-178f-11ef-b846-000c29a50cb9:1-4
            Executed_Gtid_Set: cde07bc2-178f-11ef-b846-000c29a50cb9:1-4
                

root@mysqlprod02> start replica until sql_before_gtids = 'cde07bc2-178f-11ef-b846-000c29a50cb9:40007';
Query OK, 0 rows affected (0.07 sec)

root@mysqlprod02> show replica status \G

*************************** 1. row ***************************
            Replica_IO_State: Waiting for source to send event

            Replica_IO_Running: Yes
            Replica_SQL_Running: Yes
              
            Until_Condition: SQL_BEFORE_GTIDS

            Seconds_Behind_Source: 1262

            Replica_SQL_Running_State: Waiting for dependent transaction to commit
           
            Retrieved_Gtid_Set: cde07bc2-178f-11ef-b846-000c29a50cb9:1-40008
            Executed_Gtid_Set: cde07bc2-178f-11ef-b846-000c29a50cb9:1-5027


root@mysqlprod02> show replica status \G

*************************** 1. row ***************************
            Replica_IO_State: Waiting for source to send event

            Seconds_Behind_Source: NULL

            Retrieved_Gtid_Set: cde07bc2-178f-11ef-b846-000c29a50cb9:1-40008
            Executed_Gtid_Set: cde07bc2-178f-11ef-b846-000c29a50cb9:1-40006


root@mysqlprod02> select count(*) from employeesdb.products;
+----------+
| count(*) |
+----------+
|    40000 |
+----------+
1 row in set (0.11 sec)

We can see that the table still exist in replica as we applied SQL before the drop table statement.         


In [ ]:
skipping transactions , Suppose there is an application release and in between there are 
some accidental drop table statements we need to skip and continue with other statements
=====================================================================================================

root@mysqlprod02> stop replica;
Query OK, 0 rows affected (0.00 sec)

root@mysqlprod02> show replica status \G

*************************** 1. row ***************************
             Replica_IO_State: 
                  Source_Host: 192.168.153.133
                  Source_User: replicator
        
             Replica_IO_Running: No
             Replica_SQL_Running: No


root@mysqlprod01> flush logs;
Query OK, 0 rows affected (0.08 sec)

root@mysqlprod01> show master status;
+---------------------------+----------+--------------+------------------+----------------------------------------------+
| File                      | Position | Binlog_Do_DB | Binlog_Ignore_DB | Executed_Gtid_Set                            |
+---------------------------+----------+--------------+------------------+----------------------------------------------+
| mysqlprod01_binlog.000002 |      197 |              |                  | cde07bc2-178f-11ef-b846-000c29a50cb9:1-50011 |
+---------------------------+----------+--------------+------------------+----------------------------------------------+
1 row in set (0.00 sec)

root@mysqlprod01> create database ordersdb;

create table order_details (
order_id int primary key,
order_name varchar(100),
order_date datetime
);

root@mysqlprod01> call ordersdb.sp_insertrorderdetails_data(1);
Query OK, 1 row affected (6.43 sec)

root@mysqlprod01> call ordersdb.sp_insertrorderdetails_data(10001);
Query OK, 1 row affected (6.16 sec)

root@mysqlprod01> show master status;
+---------------------------+----------+--------------+------------------+----------------------------------------------+
| File                      | Position | Binlog_Do_DB | Binlog_Ignore_DB | Executed_Gtid_Set                            |
+---------------------------+----------+--------------+------------------+----------------------------------------------+
| mysqlprod01_binlog.000002 |  8699076 |              |                  | cde07bc2-178f-11ef-b846-000c29a50cb9:1-70014 |
+---------------------------+----------+--------------+------------------+----------------------------------------------+
1 row in set (0.00 sec)

root@mysqlprod01> select count(*) from ordersdb.order_details;
+----------+
| count(*) |
+----------+
|    20000 |
+----------+
1 row in set (0.18 sec)

root@mysqlprod01> drop table ordersdb.order_details;
Query OK, 0 rows affected (0.05 sec)

root@mysqlprod01> select count(*) from ordersdb.order_details;
ERROR 1146 (42S02): Table 'ordersdb.order_details' doesn't exist
root@mysqlprod01> 
root@mysqlprod01> show master status;
+---------------------------+----------+--------------+------------------+----------------------------------------------+
| File                      | Position | Binlog_Do_DB | Binlog_Ignore_DB | Executed_Gtid_Set                            |
+---------------------------+----------+--------------+------------------+----------------------------------------------+
| mysqlprod01_binlog.000002 |  8699299 |              |                  | cde07bc2-178f-11ef-b846-000c29a50cb9:1-70015 |
+---------------------------+----------+--------------+------------------+----------------------------------------------+
1 row in set (0.00 sec)

create table order_item_details (
order_id int primary key,
order_name varchar(100),
order_date datetime
);

root@mysqlprod01> call ordersdb.sp_insertrorderitemdetails_data(1);
Query OK, 1 row affected (6.38 sec)

root@mysqlprod01> select count(*) from ordersdb.order_item_details;
+----------+
| count(*) |
+----------+
|    10000 |
+----------+
1 row in set (0.13 sec)

root@mysqlprod01> 

root@mysqlprod01> show master status;
+---------------------------+----------+--------------+------------------+----------------------------------------------+
| File                      | Position | Binlog_Do_DB | Binlog_Ignore_DB | Executed_Gtid_Set                            |
+---------------------------+----------+--------------+------------------+----------------------------------------------+
| mysqlprod01_binlog.000002 | 13087995 |              |                  | cde07bc2-178f-11ef-b846-000c29a50cb9:1-80017 |
+---------------------------+----------+--------------+------------------+----------------------------------------------+
1 row in set (0.01 sec)



Now we know that GTID 70015 is a bad one and we need to skip it in replica 
===============================================================================================

root@mysqlprod02> start replica until sql_before_gtids = 'cde07bc2-178f-11ef-b846-000c29a50cb9:70015';
Query OK, 0 rows affected (0.09 sec)


root@mysqlprod02> show replica status \G

*************************** 1. row ***************************
             Replica_IO_State: Waiting for source to send event
                  Source_Host: 192.168.153.133
                  Source_User: replicator
                  Source_Port: 3306

                  Until_Condition: SQL_BEFORE_GTIDS

                 Retrieved_Gtid_Set: cde07bc2-178f-11ef-b846-000c29a50cb9:1-80017
                 Executed_Gtid_Set: cde07bc2-178f-11ef-b846-000c29a50cb9:1-56258

root@mysqlprod02> show replica status \G

*************************** 1. row ***************************
             Replica_IO_State: Waiting for source to send event
             Source_Host: 192.168.153.133
    
             Replica_IO_Running: Yes
             Replica_SQL_Running: No
              
             Until_Condition: SQL_BEFORE_GTIDS

             Retrieved_Gtid_Set: cde07bc2-178f-11ef-b846-000c29a50cb9:1-80017
             Executed_Gtid_Set: cde07bc2-178f-11ef-b846-000c29a50cb9:1-70014


root@mysqlprod02> select count(*) from ordersdb.order_details;
+----------+
| count(*) |
+----------+
|    20000 |
+----------+
1 row in set (0.15 sec)


Run an empty transaction in place of drop statement gtid in replica as below 
==================================================================================

root@mysqlprod02> stop replica;
Query OK, 0 rows affected (0.00 sec)

root@mysqlprod02> SET GTID_NEXT='cde07bc2-178f-11ef-b846-000c29a50cb9:70015';
Query OK, 0 rows affected (0.00 sec)

root@mysqlprod02> BEGIN;
Query OK, 0 rows affected (0.00 sec)

root@mysqlprod02> COMMIT;
Query OK, 0 rows affected (0.20 sec)

root@mysqlprod02> SET GTID_NEXT='AUTOMATIC';
Query OK, 0 rows affected (0.00 sec)

root@mysqlprod02> start replica;
Query OK, 0 rows affected (0.27 sec)

root@mysqlprod02> show replica status \G
*************************** 1. row ***************************
             Replica_IO_State: Waiting for source to send event
                  Source_Host: 192.168.153.133

root@mysqlprod02> show replica status \G
*************************** 1. row ***************************
             Replica_IO_State: Waiting for source to send event
             Source_Host: 192.168.153.133
    
             Replica_IO_Running: Yes
             Replica_SQL_Running: Yes

             Retrieved_Gtid_Set: cde07bc2-178f-11ef-b846-000c29a50cb9:1-80017
             Executed_Gtid_Set: cde07bc2-178f-11ef-b846-000c29a50cb9:1-80017


root@mysqlprod02> select count(*) from ordersdb.order_details;
+----------+
| count(*) |
+----------+
|    20000 |
+----------+
1 row in set (0.14 sec)

root@mysqlprod02> select count(*) from ordersdb.order_item_details;
+----------+
| count(*) |
+----------+
|    10000 |
+----------+
1 row in set (0.14 sec)

We can see both the tables are present which are executed during the release.


In [ ]:

Primary - Replica Switchover 
=======================================================


in primary 
---------------------

root@mysqlprod01> show replicas;
+-----------+------+------+-----------+--------------------------------------+
| Server_Id | Host | Port | Source_Id | Replica_UUID                         |
+-----------+------+------+-----------+--------------------------------------+
|         2 |      | 3306 |         1 | c64bbe27-185b-11ef-9f43-000c297f8903 |
+-----------+------+------+-----------+--------------------------------------+
1 row in set (0.00 sec)


in replica 
-----------------------

root@mysqlprod02> stop replica;
Query OK, 0 rows affected (0.01 sec)

root@mysqlprod02> reset slave all;
Query OK, 0 rows affected, 1 warning (0.08 sec)

root@mysqlprod02> show replica status \G
Empty set (0.00 sec)


in primary 
-------------------------------

root@mysqlprod01> show replicas;
Empty set (0.00 sec)


in slave 
----------------------------------

[root@mysqlprod02 ~]# vi /etc/replication.cnf 
[root@mysqlprod02 ~]# 
[root@mysqlprod02 ~]# 
[root@mysqlprod02 ~]# cat /etc/replication.cnf 
[mysqld]

log-bin = /var/lib/mysql/mysqlprod02_binlog
log-bin-index = /var/lib/mysql/mysqlprod02_binlog.index

binlog-format = MIXED

gtid-mode = ON
enforce-gtid-consistency = ON

server-id = 2
[root@mysqlprod02 ~]# 

[root@mysqlprod02 ~]# systemctl stop mysqld
[root@mysqlprod02 ~]# 
[root@mysqlprod02 ~]# systemctl start mysqld
[root@mysqlprod02 ~]# 

root@mysqlprod02> show master status;
+---------------------------+----------+--------------+------------------+----------------------------------------------+
| File                      | Position | Binlog_Do_DB | Binlog_Ignore_DB | Executed_Gtid_Set                            |
+---------------------------+----------+--------------+------------------+----------------------------------------------+
| mysqlprod02_binlog.000003 |      197 |              |                  | cde07bc2-178f-11ef-b846-000c29a50cb9:1-80017 |
+---------------------------+----------+--------------+------------------+----------------------------------------------+
1 row in set (0.01 sec)

root@mysqlprod02> reset master;
Query OK, 0 rows affected (0.01 sec)

root@mysqlprod02> reset master status;
ERROR 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'status' at line 1
root@mysqlprod02> show master status;
+---------------------------+----------+--------------+------------------+-------------------+
| File                      | Position | Binlog_Do_DB | Binlog_Ignore_DB | Executed_Gtid_Set |
+---------------------------+----------+--------------+------------------+-------------------+
| mysqlprod02_binlog.000001 |      157 |              |                  |                   |
+---------------------------+----------+--------------+------------------+-------------------+
1 row in set (0.00 sec)


in primary 
-------------------------

[root@mysqlprod01 ~]# vi /var/lib/mysql-files/replication.cnf 
[root@mysqlprod01 ~]# 
[root@mysqlprod01 ~]# cat /var/lib/mysql-files/replication.cnf
[mysqld]

log-bin = /var/lib/mysql/binlog/mysqlprod01_binlog
log-bin-index = /var/lib/mysql/binlog/mysqlprod01_binlog.index
relay-log = /var/lib/mysql/relaylog/mysqlprod01_relaylog
relay-log-index = /var/lib/mysql/relaylog/mysqlprod01_relaylog.index

binlog-format = MIXED

gtid-mode = ON
enforce-gtid-consistency = ON

skip-replica-start

read-only
[root@mysqlprod01 ~]# 

[root@mysqlprod01 mysql]# mkdir relaylog
[root@mysqlprod01 mysql]# 
[root@mysqlprod01 mysql]# chown -R mysql:mysql relaylog
[root@mysqlprod01 mysql]# 

[root@mysqlprod01 mysql]# systemctl stop mysqld
[root@mysqlprod01 mysql]# 
[root@mysqlprod01 mysql]# systemctl start mysqld
[root@mysqlprod01 mysql]# 


root@mysqlprod01> change replication source to
    -> source_host = '192.168.153.134',
    -> source_port = 3306,
    -> source_user = 'replicator',
    -> source_password = 'P@ssword2024',
    -> source_auto_position = 1;
Query OK, 0 rows affected, 2 warnings (0.08 sec)

root@mysqlprod01> start replica;
Query OK, 0 rows affected (0.01 sec)

root@mysqlprod01> show replica status \G
*************************** 1. row ***************************
             Replica_IO_State: Waiting for source to send event
                  Source_Host: 192.168.153.134
                  Source_User: replicator
                  Source_Port: 3306
                Connect_Retry: 60
              Source_Log_File: mysqlprod02_binlog.000001
          Read_Source_Log_Pos: 157
               Relay_Log_File: mysqlprod01_relaylog.000002
                Relay_Log_Pos: 391
        Relay_Source_Log_File: mysqlprod02_binlog.000001
           Replica_IO_Running: Yes
          Replica_SQL_Running: Yes


in new primary ( old replica )
--------------------------------------------------------------------

root@mysqlprod02> show replicas;
+-----------+------+------+-----------+--------------------------------------+
| Server_Id | Host | Port | Source_Id | Replica_UUID                         |
+-----------+------+------+-----------+--------------------------------------+
|         1 |      | 3306 |         2 | cde07bc2-178f-11ef-b846-000c29a50cb9 |
+-----------+------+------+-----------+--------------------------------------+
1 row in set (0.00 sec)

root@mysqlprod02> create database productsdb;
Query OK, 1 row affected (0.01 sec)

root@mysqlprod02> use productsdb;
Database changed
root@mysqlprod02> create table prod_details ( prod_id int , prod_name varchar(20) , prod_date datetime );
Query OK, 0 rows affected (0.08 sec)

root@mysqlprod02> insert into prod_details values ( 10001 , 'jgcddv' , now() ),
    -> ( 10002 , 'uytrwr' , now() ),
    -> ( 10003 , 'uyrrjg' , now() );
Query OK, 3 rows affected (0.02 sec)
Records: 3  Duplicates: 0  Warnings: 0

root@mysqlprod02> select count(*) from productsdb.prod_details;
+----------+
| count(*) |
+----------+
|        3 |
+----------+
1 row in set (0.13 sec)


in new replica ( old primary )
----------------------------------------------------------------

root@mysqlprod01> show replica status \G
*************************** 1. row ***************************
             Replica_IO_State: Waiting for source to send event
                  Source_Host: 192.168.153.134
                  Source_User: replicator
                  Source_Port: 3306
                Connect_Retry: 60
              Source_Log_File: mysqlprod02_binlog.000001
          Read_Source_Log_Pos: 1054
               Relay_Log_File: mysqlprod01_relaylog.000002
                Relay_Log_Pos: 1288
        Relay_Source_Log_File: mysqlprod02_binlog.000001
           Replica_IO_Running: Yes
          Replica_SQL_Running: Yes


root@mysqlprod01> select count(*) from productsdb.prod_details;
+----------+
| count(*) |
+----------+
|        3 |
+----------+
1 row in set (0.13 sec)
